<a href="https://colab.research.google.com/github/BeniwalAnkit/Semantic-Semalarity/blob/main/Enterpret_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
eval_df = pd.read_excel("evaluation.xlsx")
train_df = pd.read_excel("train.xlsx")

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.9 MB/s eta 0:00:0

In [ ]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import EvalPrediction
from sklearn.metrics import precision_recall_fscore_support, classification_report
from datasets import Dataset
import random

In [ ]:
# For generating negative samples

def generate_negative_samples(dataset, num_negative_samples):
    negative_samples = []
    positive_examples = dataset[dataset['label'] == 1]
    
    for _ in range(num_negative_samples):
        # Randomly select a positive example
        positive_example = random.choice(positive_examples.values)
        
        # Randomly select a different reason
        negative_reason = random.choice(positive_examples['reason'].values)
        
        # Create a negative example
        negative_sample = list(positive_example[:1]) + [negative_reason, 0]
        negative_samples.append(negative_sample)
    
    return pd.DataFrame(negative_samples, columns=dataset.columns)

In [ ]:
def preprocess_dataset(df, tokenizer):
    def encode(example):
        inputs = tokenizer(example['text'], example['reason'], padding=True, truncation=True, max_length=512, return_tensors='pt')
        return {k: v.squeeze(0) for k, v in inputs.items()}
    
    dataset = Dataset.from_pandas(df)
    dataset = dataset.map(encode, batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    return dataset


In [ ]:
def compute_metrics(eval_pred: EvalPrediction):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {'precision': precision, 'recall': recall, 'f1': f1}


In [ ]:
# Generate 2061 negative samples
num_negative_samples = 2061
negative_dataset = generate_negative_samples(train_df, num_negative_samples)

# Concatenate positive and negative datasets
new_traindf = pd.concat([train_df, negative_dataset], ignore_index=True)

# Shuffle the new dataset
new_traindf = new_traindf.sample(frac=1).reset_index(drop=True)

In [ ]:
pip install --upgrade accelerate

In [ ]:
!pip install torch
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.29.1
Uninstalling transformers-4.29.1:
  Successfully uninstalled transformers-4.29.1
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.1-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)


In [ ]:
models = ['bert-base-uncased', 'distilbert-base-uncased', 'roberta-base']
model_results = {}

for model_name in models:
    print(f"Training and evaluating {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    train_dataset = preprocess_dataset(new_traindf, tokenizer)
    eval_dataset = preprocess_dataset(eval_df, tokenizer)
    
    training_args = TrainingArguments(
        output_dir=f'./results/{model_name}',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        logging_dir=f'./logs/{model_name}',
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train() 

Training and evaluating bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Map:   0%|          | 0/4122 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.303300,2.059071,0.435334,0.845718,0.574793


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.303300,2.059071,0.435334,0.845718,0.574793


In [ ]:
 # Error Analysis
    predictions = trainer.predict(eval_dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    report = classification_report(eval_dataset['label'], preds, output_dict=True)
    model_results[model_name] = report

print("Error analysis:")
for model_name, report in model_results.items():
    print(f"Model: {model_name}")
    print(report)

In [ ]:
from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Load positive samples from train file
with open("train.xlsx", "r") as f:
    positive_samples = [line.strip().split("\t") for line in f]

# Generate negative samples
negative_samples = []
for text, reason in positive_samples:
    while True:
        random_text, random_reason = random.choice(positive_samples)
        if random_text != text and random_reason != reason:
            negative_samples.append([text, random_reason])
            break

# Encode samples and compute cosine similarity
X = []
y = []
for text, reason in positive_samples + negative_samples:
    encoded_text = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    encoded_reason = tokenizer(reason, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    with torch.no_grad():
        text_embeddings = model(**encoded_text).last_hidden_state.mean(dim=1)
        reason_embeddings = model(**encoded_reason).last_hidden_state.mean(dim=1)
        similarity = F.cosine_similarity(text_embeddings, reason_embeddings)
    X.append(similarity.numpy())
    y.append(1 if reason in positive_samples else 0)

# Train logistic regression model
clf = LogisticRegression(random_state=0)
clf.fit(X, y)

# Load evaluation file and make predictions
with open("evaluation.xlsx", "r") as f:
    eval_samples = [line.strip().split("\t") for line in f]

X_eval = []
y_true = []
for text, reason in eval_samples:
    encoded_text = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    encoded_reason = tokenizer(reason, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    with torch.no_grad():
        text_embeddings = model(**encoded_text).last_hidden_state.mean(dim=1)
        reason_embeddings = model(**encoded_reason).last_hidden_state.mean(dim=1)
        similarity = F.cosine_similarity(text_embeddings, reason_embeddings)
    X_eval.append(similarity.numpy())
    y_true.append(1 if reason in positive_samples else 0)

y_pred = clf.predict(X_eval)

# Evaluate model
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy: {acc:.2f}")
print(f"Precision: {prec:.2f}")
print(f"Recall: {rec:.2f}")
print(f"F1 score: {f1:.2f}")
